## Checking for mortality!

This script loads my labelled dataset and the patient info dataset and finds patients who died within 1 day of being taken off the ventilator

#### import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import psutil
import os

#### Get ventilated patients data

In [13]:
df = pd.read_csv('sample_vents.csv')

In [14]:
df

,hadm_id,endtime,value,valueuom,re_intub_class
0,25015072,2152-04-10 18:57:00,89.116667,hour,0
1,28038802,2185-12-20 09:59:00,108.800000,hour,0
2,21790335,2140-03-11 13:44:00,33.000000,hour,0
3,24357541,2177-02-10 14:00:00,659.416667,hour,0
4,23255460,2172-11-20 08:06:00,56.216667,hour,0
...,...,...,...,...,...
24674,29960248,2146-12-09 14:23:00,4408.000000,min,1
24675,29962016,2135-10-30 11:45:00,13085.000000,min,1
24676,29974575,2131-03-03 19:54:00,5934.000000,min,1
24677,29987115,2148-02-19 10:00:00,2640.000000,min,1


#### Get admissions data

In [15]:
admits = pd.read_csv('admissions.csv')

In [16]:
admits

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag
0,12427812,21593330,2184-01-06 11:51:00,2184-01-10 11:45:00,NaN,URGENT,PHYSICIAN REFERRAL,HOME,Other,ENGLISH,NaN,UNKNOWN,NaN,NaN,0
1,14029832,22059088,2120-01-18 01:28:00,2120-01-20 16:13:00,NaN,URGENT,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,NaN,OTHER,NaN,NaN,0
2,14495017,22484010,2175-01-28 15:41:00,2175-01-29 16:00:00,NaN,DIRECT EMER.,PHYSICIAN REFERRAL,HOME,Other,?,NaN,WHITE,NaN,NaN,0
3,13676048,23865469,2193-01-19 05:27:00,2193-01-24 18:59:00,NaN,URGENT,PHYSICIAN REFERRAL,HOME,Other,?,MARRIED,WHITE,NaN,NaN,0
4,13831972,27763544,2131-01-27 04:03:00,2131-01-27 05:39:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Medicaid,ENGLISH,SINGLE,WHITE,2131-01-26 22:19:00,2131-01-27 05:39:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524515,17892964,20786062,2180-09-17 00:00:00,2180-09-18 13:37:00,NaN,SURGICAL SAME DAY ADMISSION,PHYSICIAN REFERRAL,HOME,Medicare,ENGLISH,SINGLE,WHITE,NaN,NaN,0
524516,17137572,20943099,2147-08-01 17:41:00,2147-08-02 17:30:00,NaN,EW EMER.,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,DIVORCED,HISPANIC/LATINO,2147-07-31 23:55:00,2147-08-01 19:37:00,0
524517,19389857,23176714,2189-03-01 00:58:00,2189-03-02 15:22:00,NaN,SURGICAL SAME DAY ADMISSION,PHYSICIAN REFERRAL,HOME,Other,ENGLISH,MARRIED,WHITE,NaN,NaN,0
524518,12298845,22347500,2138-05-31 00:00:00,2138-06-04 16:50:00,NaN,SURGICAL SAME DAY ADMISSION,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,NaN,NaN,0


#### Join ventilated patients df and admissions table together

In [17]:
merged_inner = pd.merge(left=df, right=admits, left_on='hadm_id', right_on='hadm_id')
merged_inner

#### Find patients that died within 24 hours of being taken off the ventilator

In [22]:
merged_inner['endtime'] = pd.to_datetime(merged_inner['endtime'])  # convert to datetime object
merged_inner['deathtime'] = pd.to_datetime(merged_inner['deathtime'])
merged_inner['time_diff']= merged_inner['endtime']-merged_inner['deathtime']  # find time difference between death and being taken off vent

In [24]:
merged_inner['timediffhour'] = merged_inner['time_diff'] / np.timedelta64(1, 'h') # convert time to hours

In [26]:
dead_patients = merged_inner[merged_inner['timediffhour'].notnull()]  # find patients that died

In [29]:
died_on_vent = dead_patients[(dead_patients['timediffhour']<24)&(dead_patients['timediffhour']>-24)] # find patients that died while on the ventilator

In [30]:
died_on_vent

,hadm_id,endtime,value,valueuom,re_intub_class,subject_id,admittime,dischtime,deathtime,admission_type,...,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag,time_diff,timediffhour
0,25015072,2152-04-10 18:57:00,89.116667,hour,0,13859862,2152-03-27 12:44:00,2152-04-11 03:00:00,2152-04-11 03:00:00,EW EMER.,...,DIED,Medicare,ENGLISH,WIDOWED,WHITE,2152-03-27 09:46:00,2152-03-27 14:17:00,1,-1 days +15:57:00,-8.050000
4,23255460,2172-11-20 08:06:00,56.216667,hour,0,19350792,2172-11-17 17:21:00,2172-11-20 06:00:00,2172-11-20 06:00:00,EW EMER.,...,DIED,Medicare,?,SINGLE,ASIAN,2172-11-17 15:05:00,2172-11-17 18:39:00,1,0 days 02:06:00,2.100000
6,22358755,2178-03-03 17:33:00,294.583333,hour,0,11434633,2178-02-17 22:38:00,2178-03-03 16:00:00,2178-03-03 16:00:00,EW EMER.,...,DIED,Medicare,ENGLISH,WIDOWED,WHITE,2178-02-17 13:42:00,2178-02-18 00:07:00,1,0 days 01:33:00,1.550000
9,24828158,2167-02-16 03:58:00,5698.000000,min,0,12570801,2167-02-06 11:59:00,2167-02-16 02:37:00,2167-02-16 02:37:00,EW EMER.,...,DIED,Medicare,ENGLISH,MARRIED,WHITE,NaN,NaN,1,0 days 01:21:00,1.350000
10,27876759,2169-02-19 03:58:00,5278.000000,min,0,16093826,2169-01-18 02:04:00,2169-02-19 00:15:00,2169-02-19 00:15:00,EW EMER.,...,DIED,Medicaid,ENGLISH,SINGLE,WHITE,2169-01-17 22:09:00,2169-01-18 03:44:00,1,0 days 03:43:00,3.716667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24441,28264065,2188-02-16 15:10:00,4150.000000,min,1,14457339,2188-02-13 17:43:00,2188-02-17 02:40:00,2188-02-17 02:40:00,OBSERVATION ADMIT,...,DIED,Other,?,SINGLE,HISPANIC/LATINO,NaN,NaN,1,-1 days +12:30:00,-11.500000
24507,28742097,2126-10-02 11:42:00,881.000000,min,1,18029528,2126-10-01 20:31:00,2126-10-03 00:22:00,2126-10-03 00:22:00,URGENT,...,DIED,Medicare,ENGLISH,MARRIED,UNKNOWN,NaN,NaN,1,-1 days +11:20:00,-12.666667
24609,29492382,2171-01-05 15:53:00,145.000000,min,1,10802465,2171-01-05 13:14:00,2171-01-06 03:00:00,2171-01-06 03:00:00,URGENT,...,DIED,Other,ENGLISH,MARRIED,WHITE,NaN,NaN,1,-1 days +12:53:00,-11.116667
24612,29525590,2169-02-10 14:30:00,5490.000000,min,1,10090454,2169-02-05 17:14:00,2169-02-11 02:14:00,2169-02-11 02:14:00,URGENT,...,DIED,Medicare,ENGLISH,SINGLE,WHITE,NaN,NaN,1,-1 days +12:16:00,-11.733333


In [33]:
dead_ids = died_on_vent['hadm_id'].tolist()


In [36]:
not_dead_vents = merged_inner[~merged_inner.hadm_id.isin(dead_ids)]

In [37]:
not_dead_vents

,hadm_id,endtime,value,valueuom,re_intub_class,subject_id,admittime,dischtime,deathtime,admission_type,...,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag,time_diff,timediffhour
1,28038802,2185-12-20 09:59:00,108.800000,hour,0,18917458,2185-12-15 00:17:00,2185-12-22 17:15:00,NaT,EW EMER.,...,HOSPICE,Medicaid,ENGLISH,MARRIED,WHITE,2185-12-14 23:05:00,2185-12-15 01:31:00,0,NaT,NaN
2,21790335,2140-03-11 13:44:00,33.000000,hour,0,19704964,2140-03-09 14:44:00,2140-03-15 13:00:00,NaT,URGENT,...,HOME,Other,?,MARRIED,UNABLE TO OBTAIN,NaN,NaN,0,NaT,NaN
3,24357541,2177-02-10 14:00:00,659.416667,hour,0,19004463,2176-12-16 18:11:00,2177-02-15 17:00:00,NaT,URGENT,...,REHAB,Medicare,ENGLISH,MARRIED,WHITE,NaN,NaN,0,NaT,NaN
5,22188993,2145-11-04 18:40:00,143.666667,hour,0,11538389,2145-10-29 12:46:00,2145-11-09 17:30:00,2145-11-09 17:30:00,EW EMER.,...,DIED,Medicare,ENGLISH,MARRIED,UNKNOWN,2145-10-29 11:07:00,2145-10-29 14:00:00,1,-5 days +01:10:00,-118.833333
7,21880799,2134-05-22 17:58:00,2.290278,day,0,16606203,2134-05-20 09:51:00,2134-05-22 18:00:00,NaT,EW EMER.,...,CHRONIC/LONG TERM ACUTE CARE,Medicare,?,MARRIED,HISPANIC/LATINO,2134-05-20 06:11:00,2134-05-20 11:15:00,0,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24674,29960248,2146-12-09 14:23:00,4408.000000,min,1,16691924,2146-11-30 16:09:00,2146-12-17 17:04:00,NaT,DIRECT EMER.,...,SKILLED NURSING FACILITY,Medicare,ENGLISH,WIDOWED,WHITE,NaN,NaN,0,NaT,NaN
24675,29962016,2135-10-30 11:45:00,13085.000000,min,1,16796371,2135-10-21 08:16:00,2135-11-04 13:30:00,NaT,EW EMER.,...,REHAB,Medicare,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2135-10-21 05:47:00,2135-10-21 09:45:00,0,NaT,NaN
24676,29974575,2131-03-03 19:54:00,5934.000000,min,1,10020944,2131-02-27 15:34:00,2131-03-13 17:01:00,NaT,EW EMER.,...,SKILLED NURSING FACILITY,Medicare,ENGLISH,NaN,UNKNOWN,2131-02-27 13:16:00,2131-02-27 16:40:00,0,NaT,NaN
24677,29987115,2148-02-19 10:00:00,2640.000000,min,1,19652570,2148-02-16 17:42:00,2148-02-20 19:00:00,2148-02-20 19:00:00,URGENT,...,DIED,Other,ENGLISH,NaN,UNKNOWN,NaN,NaN,1,-2 days +15:00:00,-33.000000


In [38]:
not_dead_vents.to_csv('sample_vents.csv',index= False)